# Homework 2: Graph Mining
*Due Wednesday November 14th, 2018 at 11:59 pm*

*Notebook Author: Koki Sasagawa*
 
Mining a large social network to uncover how well homophily can predict identity as well as the network structure. 

**Task1:** Link Predction - mostly complete social network that contains some large number of edges removed. Task is to predict up to 50 edges that system predicts to be most likely. 

## Data 

1. `network.tsv` - large social network specified in edge list format 
   - two columns where each column is label for a node in the graph
   - undirected graph, thus order does not matter
   - nodes assigned random numeric ID that have no meaning

## Submission (modified after announcement) 

**Link prediction** should be a text file with two items on each line: node1 and node2  
The first 50000 most likely links will be submitted. 


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import networkx as nx
import time
from pprint import pprint

## 1a. Create graph by reading the tsv file with networkx

In [2]:
print("Creating network graph...")
start_time = time.time() 

with open("../data/network.tsv", 'rb') as f:
    grph = nx.read_edgelist(path=f, delimiter='\t', encoding='utf8')

end_time = time.time()
print("Network graph created. Process took {:.04f} seconds".format(end_time - start_time))

# Check that graph is of correct size
print("Number of edges: {}".format(grph.number_of_edges())) # There should be 30915267
print("Number of nodes: {}".format(grph.number_of_nodes())) # There should be 6626753

Creating network graph...
Network graph created. Process took 302.3193 seconds
Number of edges: 30915267
Number of nodes: 6626753


## 1b. Create graph by reading the tsv file with pandas read_csv as chunks

In [7]:
# # Read data as chunks
# print("Reading network data as chunks...")
# start_time = time.time()

# g_data_chunks = pd.read_csv("./all/network.tsv",
#                             delimiter='\t',
#                             usecols=[0,1],
#                             names=['n', 'v'],
#                             dtype={'n': np.int32, 'v': np.int32},
#                             header=None,
#                             chunksize=100000)

# # Combine
# # g_data_chunks pd.concat(g_data_chunks)

# end_time = time.time()
# print("Chunks created. Process took {:.04f} seconds".format(end_time - start_time))

In [8]:
# print("Creating network graph...")

# start_time = time.time() 

# # Initalize undirected simple graph
# grph = nx.Graph()

# # Populate the graph by adding edges from chunked dataframes
# for chunk in g_data_chunks: 
#   grph.add_edges_from([tuple(x) for x in chunk.values])

# end_time = time.time()

# print("Network graph created. Process took {:.04f} seconds".format(end_time - start_time))

# # Check that graph is of correct size
# print("Number of edges: {}".format(grph.number_of_edges())) # There should be 30915267
# print("Number of nodes: {}".format(grph.number_of_nodes())) # There should be 6626753

## 2. Generate an adjacency list and save it as a text file

In [ ]:
# Taken and modified from stack overflow: https://stackoverflow.com/
# questions/34917550/write-a-graph-into-a-file-in-an-adjacency-list
# -form-mentioning-all-neighbors-of
def adj_list_to_file(G, file_name):
    '''Create adjacency list and save as text file

    :params G: network graph
    :type G: networkx graph
    :params file_name: file name and save location
    :type file_name: str
    '''

    with open(file_name, "w") as f:
        for n in G.nodes():
            f.write(str(n) + ',')
            for neighbor in G.neighbors(n):
                f.write(str(neighbor) + ' ')
            f.write('\n')

In [ ]:
# Write save save adjacency list 
adj_list_to_file(grph, '../data/adjacency_list.txt')

## 3. Read the adjacency list file and create a dictionary from it 

In [2]:
# Open file and create dictionary
adj_list = {}

with open('../data/adjacency_list.txt', 'r') as f:
    # For each line in the file, create a dictionary that has a key = node and value = edges
    for line in f:
        adj_list[line.split(',')[0]] = line.split(',')[1].rstrip().split(' ')

## 4. The following functions are for the fast algorithm for Common Neighbors Filtering

In [2]:
def filter_by_lemma1(adj_list, L):
    '''Filter nodes with neighbors less than or equal to L

    If the number of their neighbors is no greater than threshold L,
    these pairs will not have more than L common neighbors.

    For example, the following is an adjacency list represented as a
    python dictionary. The key is a node, and the value is a list of
    common neighbors.

    {0: [1, 2, 4, 5, 7],
     1: [0, 2, 4, 7],
     2: [0, 1, 3, 5, 6],
     3: [2, 4, 6, 7],
     4: [0, 1, 3, 6],
     5: [0, 2],
     6: [2, 3, 4],
     7: [0, 1, 3]}

    Setting L to 3, this function will filter out nodes that have
    3 or less neighbors. The resulting nodes will be the following:

    {0: [1, 2, 4, 5, 7],
     1: [0, 2, 4, 7],
     2: [0, 1, 3, 5, 6],
     3: [2, 4, 6, 7],
     4: [0, 1, 3, 6]}

    :param adj_list: adjacency list
    :type adj_list: dict
    :param L: threshold for common neighbors
    :type L: int
    :return: adjacency list containing nodes with more than L neighbors
    :rtype: dict
    '''

    adj_new = {}

    for k, v in adj_list.items():
        if len(v) > L:
            adj_new[k] = v

    return adj_new

In [3]:
def invert_adjacency_list(adj_list):
    '''Invert the adjacency matrix.

    For example, the following is an adjacency list represented as a
    python dictionary. The key is a node, and the value is a list of
    common neighbors of that node.

    {0: [1, 2, 4, 5, 7],
     1: [0, 2, 4, 7],
     2: [0, 1, 3, 5, 6],
     3: [2, 4, 6, 7],
     4: [0, 1, 3, 6]}

    Starting with node 1, we see that it appears in the adjacency
    list of node 0, 2, and 4. Thus, the inverted representation will
    be '1: [0, 2, 4]'. The resulting inverted adjaceny list will be
    the following:

    {0: [1, 2, 4],
     1: [0, 2, 4],
     2: [0, 1, 3],
     3: [2, 4],
     4: [0, 1, 3],
     5: [0, 2],
     6: [2, 3, 4],
     7: [0, 1, 3]}

    :param adj_list: adjacency list
    :type adj_list: dict
    :return: inverted adjacency list
    :rtype: dict
    '''

    adj_inv = {}

    for k, v in adj_list.items():
        for i in v:
            if i in adj_inv:
                adj_inv[i].append(k)
            else:
                adj_inv[i] = [k]

    return adj_inv

In [4]:
def generate_accompanied_groups(adj_list):
    '''Generate accompanying groups in (address, size) representation

    For example, node 4 is present in the adjacency list of node 0.

    {0: [1, 2, 4],
     1: [0, 2, 4],
     2: [0, 1, 3],
     3: [2, 4],
     4: [0, 1, 3],
     5: [0, 2],
     6: [2, 3, 4],
     7: [0, 1, 3]}

    Since this is the adjacency list of node 0, the address is 0. The
    ranking of a node is equal to the size of the accompanied group.
    For node 4, the size of the accompanying group '[1, 2]' is 2. The
    accompanied group in the address, size representation would be
    '{4: (0, 2)}'. The resulting accompanied groups will be the
    following:

    {1: [(2, 1), (4, 1), (7, 1)],
     2: [(1, 1), (5, 1), (0, 1)],
     3: [(2, 2), (4, 2), (7, 2), (6, 1)],
     4: [(1, 2), (0, 2), (3, 1), (6, 2)]}

    :param adj_list: inverted adjacency list
    :type adj_list: dict
    :param L: threshold for common neighbors
    :type L: int
    :return: accompanied groups in (adress, size) representation
    :rtype: dict
    '''

    acc_group = {}

    # Find accompanied groups of nodes by address and size
    for k, v in adj_list.items():
        for i in range(1, len(v)):
            if v[i] in acc_group:
                acc_group[v[i]].append((k, i))
            else:
                acc_group[v[i]] = [(k, i)]

    return acc_group

In [5]:
def filter_by_lemma2(acc_group, L):
    '''Filter accompanying groups less than or equal to L

    After filtering by lemma 1, if node 'u' appears at most in L
    node adjacencies (having no greater than L accompanied groups),
    the common neighbor of any node pair containing 'u' will be no
    greater than L.

    For example, the following is a python dictionary representing
    the accompanied groups of a network.

    {1: [(2, 1), (4, 1), (7, 1)],
     2: [(1, 1), (5, 1), (0, 1)],
     3: [(2, 2), (4, 2), (7, 2), (6, 1)],
     4: [(1, 2), (0, 2), (3, 1), (6, 2)]}

    Setting L to 3, this function will filter out nodes that have
    3 or less groups. The resulting accompanied groups will be the
    following:

    {3: [(2, 2), (4, 2), (7, 2), (6, 1)],
     4: [(1, 2), (0, 2), (3, 1), (6, 2)]}

    :param acc_group: accompanied groups
    :type acc_group: dict
    :param L: threshold for common neighbors
    :type L: int
    :return: accompanied groups greater than L
    :rtype: dict
    '''

    f_acc_group = {}

    # Filter by L
    for k, v in acc_group.items():
        if len(v) > L:
            f_acc_group[k] = v

    return f_acc_group

In [6]:
def generate_node_pairs(acc_group, adj_list, L):
    '''Generate node pairs with CN greater than L.

    For example, the following is a python dictionary representing
    the accompanied groups of a network.

    {3: [(2, 2), (4, 2), (7, 2), (6, 1)],
     4: [(1, 2), (0, 2), (3, 1), (6, 2)]}

    Rember that the accompanied groups are in (address, size) format.
    Looking at the first group for node 4, the address is pointing to
    adjacency list 1, and the size is 2. Take a look at the following
    adjacency list:

    {0: [1, 2, 4],
     1: [0, 2, 4],
     2: [0, 1, 3],
     3: [2, 4],
     4: [0, 1, 3],
     5: [0, 2],
     6: [2, 3, 4],
     7: [0, 1, 3]}

    Looking at node 1's adjacency list, the 2 other nodes accompanying
    node 4 are 0 and 2. This means that node 0, 2, and 4 share the same
    common neighbor node 1. If we look up the address and size for all
    groups and calculate the total common neighbors shared, we get the
    following:

       |_0_|_1_|_2_|_3_|
     3 | 3 | 2 | 2 |   |
     4 | 1 | 1 | 1 | 1 |

    :param acc_group: accompanied groups
    :type acc_group: dict
    :param adj_list: inverted adjacency list
    :type adj_list: dict
    :return: node pairs and CN values
    :rtype: list
    '''

    node_pairs = {}

    for k, v in acc_group.items():
        for i in v:
            # Read adjaceny list up to size (rank)
            for j in adj_list[i[0]][:i[1]]:
                node_pairs[(k, j)] = node_pairs.get((k, j), 0) + 1

    filtered_node_pairs = []

    for k, v in node_pairs.items():
        if v > L:
            filtered_node_pairs.append((k, v))

    return filtered_node_pairs

## TEST that the functions work properly 

In [7]:
import unittest 

class TestFilter(unittest.TestCase):
    
    # Run before every single test
    def setUp(self):
        self.threshold = 3
        
        self.adj_list_1 = {0: [1, 2, 4, 5, 7],
                           1: [0, 2, 4, 7],
                           2: [0, 1, 3, 5, 6],
                           3: [2, 4, 6, 7],
                           4: [0, 1, 3, 6],
                           5: [0, 2],
                           6: [2, 3, 4],
                           7: [0, 1, 3]}
        
        self.adj_list_2 = {0: [1, 2, 4, 5, 7], 
                           1: [0, 2, 4, 7], 
                           2: [0, 1, 3, 5, 6], 
                           3: [2, 4, 6, 7], 
                           4: [0, 1, 3, 6]}
                            
        self.inv_list = {0: [1, 2, 4],
                         1: [0, 2, 4],
                         2: [0, 1, 3],
                         3: [2, 4],
                         4: [0, 1, 3],
                         5: [0, 2],
                         6: [2, 3, 4],
                         7: [0, 1, 3]}
        
        self.acc_group_1 = {1: [(2, 1), (4, 1), (7, 1)],
                            2: [(1, 1), (5, 1), (0, 1)],
                            3: [(2, 2), (4, 2), (7, 2), (6, 1)],
                            4: [(1, 2), (0, 2), (3, 1), (6, 2)]}
        
        self.acc_group_2 = {3: [(2, 2), (4, 2), (7, 2), (6, 1)], 
                            4: [(1, 2), (0, 2), (3, 1), (6, 2)]}
        
        self.pair_node = [((4, 2), 4)]
    
    def test_filter_lemma1(self):
        #print("Test filter lemma 1...")
        self.assertEqual(filter_by_lemma1(self.adj_list_1, self.threshold), self.adj_list_2)
    
    def test_inverted_adjacency_list(self):
        #print("Test inverted_adjacency...")
        self.assertEqual(invert_adjacency_list(self.adj_list_2), self.inv_list)
    
    def test_generate_accompanied_groups(self):
        # print("Test both lemma filters...")
        self.assertCountEqual(generate_accompanied_groups(self.inv_list), self.acc_group_1)
        
    def test_filter_lemma2(self):
        self.assertEqual(filter_by_lemma2(self.acc_group_1, self.threshold), self.acc_group_2)
    
    def test_pair_node(self):
        self.assertEqual(generate_node_pairs(self.acc_group_2, self.inv_list, self.threshold), self.pair_node)

In [8]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.005s

OK


## Find Optimal L value 

In [ ]:
# Threshold. Count candidates from 5 to 100 (increments of 5)
# Count how many node pairs are above a certain threshold 
results = {}

for L in range(0, 150, 1):
    count = 0
    for k, v in adj_list.items():
        if len(v) > L:
            count += 1
    results[L] = count  

In [ ]:
node_count = pd.DataFrame.from_dict({'L':list(results.keys()), 'node_count': list(results.values())})

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Set the style of seaborn plot
sns.set(style='darkgrid', palette='husl', font_scale=1.5)

# Create matplotlib Figure and Axes object
f, ax = plt.subplots(figsize=(15,9))

# Create the plot
g = sns.lineplot(x='L', y='node_count', markers=True, data=node_count)

# Fine Tuning font size
g.set_xlabel('L', fontsize=20)
g.set_ylabel('Number of Nodes', fontsize=20)
g.axes.set_title('Threshold to number of nodes', fontsize=30)

# Display
plt.show(g)

## Generate candidate node pairs

In [41]:
# Define Threshold
L = 50

print("Step 1: Filter adjacency list")
f_adj_list = filter_by_lemma1(adj_list, L)

print("Step 2: Invert adjacency list")
inv_adj_list = invert_adjacency_list(f_adj_list)

# Clear Variables
f_adj_list = None

print("Step 3: Create accompanied groups")
acc_groups = generate_accompanied_groups(inv_adj_list)

print("Step 4: Filter accompanied groups")
f_acc_groups = filter_by_lemma2(acc_groups, L)

# Clear variables
acc_groups = None

candidate_node_pairs = generate_node_pairs(f_acc_groups, inv_adj_list, L)
print("Candidate node pairs generated!")

# Clear variables
f_acc_groups = None
inv_adj_list = None

## Save candidate ndoes as csv file

In [64]:
def save_candidate_pairs(c_pairs, file_name):
    '''Save candidate pairs as csv file

    :params c_pairs: candidate node pairs
    :type c_pairs: nested list
    :params file_name: file name and save location
    :type file_name: str
    '''

    with open(file_name, "w") as f:
        f.write('node1,node2,CN\n')
        for i in c_pairs:
            f.write('{}, {}, {}\n'.format(i[0][0], i[0][1], i[1]))

In [65]:
# Save
save_candidate_pairs(candidate_node_pairs, '../data/candidate_pairs.csv')

## Resulting nodes are the node pairs that have common neighbors above a certain threshold. From this list, for nodes that are not yet already connected, find the 50,000 most likely pairs. 

In [3]:
# Open candidate nodes
with open('../data/candidate_pairs.csv', 'r') as f:
    # For each line in the file, create a dictionary that has a key = node and value = edges
    for line in f:
        n1, n2, v = line.rstrip('\n').split(',')
        break

In [4]:
candidate_nodes = pd.read_csv('../data/candidate_pairs.csv',
                              dtype={'node1': np.int32, 'node2': np.int32, 'CN': np.int32})

# Check
print(candidate_nodes.shape[0])
candidate_nodes.head()

231167


,node1,node2,CN
0,1091804,967845,67
1,1091804,1354523,51
2,1091804,2309755,68
3,4573414,967845,101
4,4573414,1354523,54


In [5]:
# Sort by CN value
candidate_nodes.sort_values('CN', inplace=True, ascending=False)
candidate_nodes.head()

,node1,node2,CN
140294,6247815,1542000,593
123751,5401224,988315,557
3661,304552,6314086,545
123761,5401224,885348,521
123748,5401224,4930496,509


In [49]:
def link_prediction(c_pairs, adj_list, limit):
    '''Infer the most likely links between nodes in static network

    This function infers the mostly likely links between nodes
    based on the number of common neighbors.

    :params adj_list: adjacency list
    :type adj_list: dict
    :params c_pairs: candidate node pairs
    :type c_pairs: pandas dataframe
    :params limit: limit number of results returned
    :type limit: int
    :return: predicted links between nodes
    :rtype: list
    '''

    predictions = []

    count = 0

    for i, r in c_pairs.iterrows():
        if count <= limit:
            # keys in the adj_list dict are string type
            if str(r[1]) in adj_list.get(str(r[0]), 0):
                continue
            else:
                predictions.append((r[0], r[1]))
                count += 1

    return predictions

In [50]:
predicted_links = link_prediction(adj_list, candidate_nodes, 50000)

In [51]:
def save_predicted_links(predicted_links, file_name):
    with open(file_name, "w") as f:
        for i in predicted_links:
            f.write('{} {}\n'.format(i[0], i[1]))

In [52]:
# Save 
save_predicted_links(predicted_links, '../data/predicted_links.txt')

## Test that the output is a novel prediction

In [53]:
# Check that results don't exist in adj_list already
# In other words, they are newly infered links

for i in predicted_links:
    if str(i[1]) in adj_list.get(str(i[0]), 0):
        print('Link between {} and {} already exists! This link is not a valid prediction.'.format(i[0], i[1]))